In [2]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple
import os

In [3]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder {folder_path} created successfully.")
    else:
        print(f"Folder {folder_path} already exists.")


create_folder_if_not_exists("plots")
create_folder_if_not_exists("errors")

Folder plots already exists.
Folder errors already exists.


In [4]:
import csv


def save_to_csv(filename, data):
    filename = "errors/" + filename + ".csv"
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)

In [24]:
def multiply_matrices(A,B):
    result = np.zeros((len(A), len(B[0])), dtype=np.float64)
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                result[i][j] += A[i][k] * B[k][j]
    return result

def residual_stop_criterium(A, x, b, rho):
    def norm(vector):
        max_value = -np.inf
        for value in vector:
            if np.abs(value) > max_value:
                max_value = np.abs(value)
        return max_value

    return norm(multiply_matrices(A, x) - b) < rho


def difference_stop_criterium(x, x_new, rho):
    def norm(vector):
        max_value = -np.inf
        for value in vector:
            if np.abs(value) > max_value:
                max_value = np.abs(value)
        return max_value

    return norm(x - x_new) < rho

In [ ]:
def get_random_sign():
    return np.random.choice([-1, 1])


def get_x(n, variable_type):
    x_ = np.zeros(n, dtype=variable_type)
    for i in range(n):
        x_[i] = variable_type(variable_type(get_random_sign()))
    return x_


def get_b(A, x, n, variable_type):
    b = np.zeros(n, dtype=variable_type)
    for i in range(n):
        for j in range(n):
            b[i] += A[i][j] * x[j]
        print()
    return b

In [ ]:
k = np.float64(6)
m = np.float64(4)
one = np.float64(1)


def get_A(size):
    global k, m, one
    A = np.zeros((size, size), dtype=np.float64)
    for i in range(size):
        for j in range(size):
            i_ = np.float64(i)
            j_ = np.float64(j)

            if i == j:
                A[i][j] = k
            elif j > i:
                A[i][j] = (-one) ** (j_ + one) * m / (j_ + one)
            elif j + 1 == i:
                A[i][j] = m / (i_ + one)
            else:
                A[i][j] = np.float64(0)
    return A


def get_D(n):
    global k, m, one
    D = np.zeros((n, n), dtype=np.float64)
    for i in range(n):
        D[i][i] = k
    return D


def get_D_inverted(n):
    global k, m, one
    D = np.zeros((n, n), dtype=np.float64)
    k_ = np.float64(k)
    for i in range(n):
        D[i][i] = one / k_
    return D


def get_negative_D_inverted(n):
    global k, m, one
    D = np.zeros((n, n), dtype=np.float64)
    k_ = np.float64(k)
    minus_one = np.float64(-1)
    for i in range(n):
        D[i][i] = minus_one / k_
    return D


def get_L(n):
    global k, m, one
    L = np.zeros((n, n), dtype=np.float64)
    for i in range(n):
        for j in range(n):
            i_ = np.float64(i)
            j_ = np.float64(j)
            if i == j + 1:
                L[i][j] = m / (i_ + one)
    return L


def get_U(n):
    global k, m, one
    U = np.zeros((n, n), dtype=np.float64)
    for i in range(n):
        for j in range(n):
            i_ = np.float64(i)
            j_ = np.float64(j)
            if j > i:
                U[i][j] = (-one) ** (j_ + one) * m / (j_ + one)
    return U


def get_M(n):
    U = get_U(n)
    L = get_L(n)
    negative_D_inverted = get_negative_D_inverted(n)
    M = np.dot(negative_D_inverted, L + U)
    return M


In [ ]:



class Jacobi:
    def __init__(self, stop_criterium, b, n, x_start, rho):
        self.b = b
        self.n = n
        self.x = x_start
        self.rho = rho
        self.M = get_M(n)
        self.N = get_D_inverted(n)
        self.iterations = 0
        self.stop_criterium = stop_criterium
        self.errors = []
        if stop_criterium == residual_stop_criterium:
            self.stop_criterium_chosen = "res"
        elif stop_criterium == difference_stop_criterium:
            self.stop_criterium_chosen = "diff"
        else:
            raise Exception("Wrong stop criterium")

    def calculate(self):
        next_x = np.zeros(self.n, dtype=np.float64)#%%
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple
import os

In [ ]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder {folder_path} created successfully.")
    else:
        print(f"Folder {folder_path} already exists.")


create_folder_if_not_exists("plots")
create_folder_if_not_exists("errors")

In [ ]:
import csv


def save_to_csv(filename, data):
    filename = "errors/" + filename + ".csv"
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)

In [ ]:
def residual_stop_criterium(A, x, b, rho):
    def norm(vector):
        max_value = -np.inf
        for value in vector:
            if np.abs(value) > max_value:
                max_value = np.abs(value)
        return max_value

    def multiply(A, x):
        result = np.zeros(len(A), dtype=np.float64)
        for i in range(len(A)):
            for j in range(len(A)):
                result[i] += A[i][j] * x[j]
        return result

    return norm(multiply(A, x) - b) < rho


def difference_stop_criterium(x, x_new, rho):
    def norm(vector):
        max_value = -np.inf
        for value in vector:
            if np.abs(value) > max_value:
                max_value = np.abs(value)
        return max_value

    return norm(x - x_new) < rho

In [ ]:
def spectral_radius(matrix):
    eigenvalues = np.abs(np.linalg.eigvals(matrix))
    return np.max(eigenvalues)


'''for n in range(3, 150):
    M = get_M(n)
    print(f"n = {n}, spectral radius = {spectral_radius(M)}")'''#%%
def spectral_radius(matrix):
    eigenvalues = np.abs(np.linalg.eigvals(matrix))
    return np.max(eigenvalues)


'''for n in range(3, 150):
    M = get_M(n)
    print(f"n = {n}, spectral radius = {spectral_radius(M)}")'''